In [3]:
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import nltk

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import re

In [4]:
# Esto sirve para configurar NLTK. La primera vez puede tardar un poco
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Area-
[nltk_data]     Desa15\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Area-
[nltk_data]     Desa15\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from google.colab import auth
auth.authenticate_user()  # Esto abrirá una ventana emergente para autenticarte con tu cuenta de Google.


ModuleNotFoundError: No module named 'google'

In [7]:
# Especifica la ruta del archivo Parquet en Google Cloud Storage
ruta_del_archivo_gcs = 'gs://top-vial-398602/Output-tables/Tablas_consolidadas_anuales/2021.parquet'

# Carga el archivo Parquet desde Google Cloud Storage en un DataFrame de Pandas
dataset = pd.read_parquet(ruta_del_archivo_gcs)

OSError: google::cloud::Status(UNAVAILABLE: Retry policy exhausted GetObjectMetadata: Could not create a OAuth2 access token to authenticate the request. The request was not sent, as such an access token is required to complete the request successfully. Learn more about Google Cloud authentication at https://cloud.google.com/docs/authentication. The underlying error message was: PerformWork() - CURL error [6]=Couldn't resolve host name)

In [36]:
dataset = dataset[(dataset['rating'] == 5) & (dataset['text'] != '') & (dataset['num_of_reviews'] > 2000)] # Recorte de datos

In [37]:
dataset.head()

,business_id,user_id,user_name,rating,text,datetime,año,us_state,name,address,...,hours,postal_code,num_of_reviews,attributes,ex_categories,state_name,city,plataforma,gmap_id,price
3102,1b5mnK8bMnnju_cvU65GqQ,-6bsEIMFz6ndaWXaakISSw,Jessica,5.0,For having such a massive sweet tooth and expe...,2021-03-19 20:00:51,2021,PA,Biscuit Love: Gulch,316 11th Ave S,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",37203,4207,"{'NoiseLevel': ""'average'"", 'HasTV': 'False', ...",Burgers,Pennsylvania,Nashville,Yelp,None,None
3103,1b5mnK8bMnnju_cvU65GqQ,-pZI4YwvEPXY2hzolV4mdA,Anna,5.0,Great food & service! Family friendly. Great...,2021-03-21 16:35:29,2021,PA,Biscuit Love: Gulch,316 11th Ave S,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",37203,4207,"{'NoiseLevel': ""'average'"", 'HasTV': 'False', ...",Burgers,Pennsylvania,Nashville,Yelp,None,None
3106,1b5mnK8bMnnju_cvU65GqQ,5yyECn2fVnrANj__DysArw,Chelsea,5.0,This place was so cute and so good! Cheese gri...,2021-09-18 14:47:03,2021,PA,Biscuit Love: Gulch,316 11th Ave S,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",37203,4207,"{'NoiseLevel': ""'average'"", 'HasTV': 'False', ...",Burgers,Pennsylvania,Nashville,Yelp,None,None
3107,1b5mnK8bMnnju_cvU65GqQ,7fWkv_OIDkl3sMWcZZaYog,Kenneth,5.0,"Be prepared to wait, but it is definitely wort...",2021-05-28 15:38:12,2021,PA,Biscuit Love: Gulch,316 11th Ave S,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",37203,4207,"{'NoiseLevel': ""'average'"", 'HasTV': 'False', ...",Burgers,Pennsylvania,Nashville,Yelp,None,None
3108,1b5mnK8bMnnju_cvU65GqQ,AsZjQACGglrxvPnfjwaQmA,Jasmine,5.0,Line was long but it moved fast and the food w...,2021-09-10 19:41:49,2021,PA,Biscuit Love: Gulch,316 11th Ave S,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",37203,4207,"{'NoiseLevel': ""'average'"", 'HasTV': 'False', ...",Burgers,Pennsylvania,Nashville,Yelp,None,None


In [38]:
dataset.shape

(16248, 23)

In [39]:
columnas_deseadas = ['business_id', 'rating', 'text', 'name', 'address', 'latitude', 'longitude','postal_code', 'ex_categories', 'state_name', 'city']

# Selecciona las columnas deseadas
df = dataset[columnas_deseadas]

In [40]:
df.shape

(16248, 11)

In [41]:
df = df[df['text'] != '']


In [42]:
df = df.drop_duplicates()


In [43]:
df.shape

(12443, 11)

In [44]:
df.head()

,business_id,rating,text,name,address,latitude,longitude,postal_code,ex_categories,state_name,city
3102,1b5mnK8bMnnju_cvU65GqQ,5.0,For having such a massive sweet tooth and expe...,Biscuit Love: Gulch,316 11th Ave S,36.15305,-86.783231,37203,Burgers,Pennsylvania,Nashville
3103,1b5mnK8bMnnju_cvU65GqQ,5.0,Great food & service! Family friendly. Great...,Biscuit Love: Gulch,316 11th Ave S,36.15305,-86.783231,37203,Burgers,Pennsylvania,Nashville
3106,1b5mnK8bMnnju_cvU65GqQ,5.0,This place was so cute and so good! Cheese gri...,Biscuit Love: Gulch,316 11th Ave S,36.15305,-86.783231,37203,Burgers,Pennsylvania,Nashville
3107,1b5mnK8bMnnju_cvU65GqQ,5.0,"Be prepared to wait, but it is definitely wort...",Biscuit Love: Gulch,316 11th Ave S,36.15305,-86.783231,37203,Burgers,Pennsylvania,Nashville
3108,1b5mnK8bMnnju_cvU65GqQ,5.0,Line was long but it moved fast and the food w...,Biscuit Love: Gulch,316 11th Ave S,36.15305,-86.783231,37203,Burgers,Pennsylvania,Nashville


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12443 entries, 3102 to 1266566
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   business_id    7808 non-null   object 
 1   rating         12443 non-null  float64
 2   text           12336 non-null  object 
 3   name           12443 non-null  object 
 4   address        12443 non-null  object 
 5   latitude       12443 non-null  float64
 6   longitude      12443 non-null  float64
 7   postal_code    12443 non-null  object 
 8   ex_categories  12443 non-null  object 
 9   state_name     12443 non-null  object 
 10  city           7808 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


MUESTRA DE DATOS A UTILIZAR

In [46]:
dataset=pd.concat([df.text,df.ex_categories],axis=1)
dataset.dropna(axis=0,inplace=True)  # Si hay alguna nan, tiramos esa instancia
dataset.head()

,text,ex_categories
3102,For having such a massive sweet tooth and expe...,Burgers
3103,Great food & service! Family friendly. Great...,Burgers
3106,This place was so cute and so good! Cheese gri...,Burgers
3107,"Be prepared to wait, but it is definitely wort...",Burgers
3108,Line was long but it moved fast and the food w...,Burgers


In [47]:
# Traemos nuevamente las stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [48]:
# Recorremos todos los textos y le vamos aplicando la Normalizacion y luega el Stemming a cada uno
reseña_list=[]
for reseña in dataset.text:
    # Vamos a reemplazar los caracteres que no sean leras por espacios
    reseña=re.sub("[^a-zA-Z]"," ",str(reseña))
    # Pasamos todo a minúsculas
    reseña=reseña.lower()
    # Tokenizamos para separar las palabras del titular
    reseña=nltk.word_tokenize(reseña)

    # Sacamos las Stopwords
    reseña = [palabra for palabra in reseña if not palabra in stopwords]

    ## Hasta acá Normalizamos, ahora a stemmizar

    # Aplicamos la funcion para buscar la raiz de las palabras
    reseña=[stemmer.stem(palabra) for palabra in reseña]
    # Por ultimo volvemos a unir la rreseña
    reseña=" ".join(reseña)

    # Vamos armando una lista con todos las reseñas
    reseña_list.append(reseña)


In [49]:
dataset["text_stem"] = reseña_list
dataset.tail(50)

,text,ex_categories,text_stem
1264984,(Translated by Google) Very excellent\n\n(Orig...,Southern restaurant (US),translat googl excel origin muy excel
1264992,Outdoor seating with a great atmosphere,Southern restaurant (US),outdoor seat great atmospher
1265021,Such a fun time whenever we get to visit!,Southern restaurant (US),fun time whenev get visit
1265026,Great food and drinks. Our servers at the bar...,Southern restaurant (US),great food drink server bar super nice abl bal...
1265060,Drove from w.p.b. Florida just to eat lunch at...,Southern restaurant (US),drove w p b florida eat lunch restaur food goo...
1265073,Great bbq worth the money.mac and cheese gotta...,Southern restaurant (US),great bbq worth money mac chees got ta tri
1265107,"I loved this place. Their price, food and cust...",Southern restaurant (US),love place price food custom servic fantast pl...
1265111,"Great night out with the kids, the dinner was ...",Southern restaurant (US),great night kid dinner excel waitress gem back
1265112,We have been going to House of Blues for years...,Southern restaurant (US),go hous blue year watch concert never meal yum...
1265117,My favorite music venue in Orlando.,Southern restaurant (US),favorit music venu orlando


In [50]:
dataset_stem=pd.concat([dataset.text_stem, dataset.ex_categories],axis=1)
dataset_stem.dropna(axis=0,inplace=True)  # Por si quedaron titulares vacios

In [51]:
dataset_stem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12336 entries, 3102 to 1266566
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_stem      12336 non-null  object
 1   ex_categories  12336 non-null  object
dtypes: object(2)
memory usage: 289.1+ KB


In [54]:
dataset_stem

,text_stem,ex_categories
3102,massiv sweet tooth experienc dessert mani citi...,Burgers
3103,great food servic famili friendli great coffe ...,Burgers
3106,place cute good chees grit amaz glad got got b...,Burgers
3107,prepar wait definit worth biscuit gravi egg di...,Burgers
3108,line long move fast food good biscuit hot chic...,Burgers
...,...,...
1266373,go jambalaya amaz shrimp grit good bread pud o...,Southern restaurant (US)
1266404,food delici servic friendli took us without re...,Southern restaurant (US)
1266414,ate bar wait,Southern restaurant (US)
1266510,translat googl like lot spectacular place wond...,Southern restaurant (US)


In [62]:
def get_recommendations_by_terms(df, query_terms, num_recommendations=10):
    recommendations = []

    for index, row in df.iterrows():
        content_terms = set(row['text_stem'].split())  # Suponiendo que 'content' contiene las palabras clave del contenido

        # Calcula la similitud de Jaccard entre los términos de consulta y los términos del contenido
        jaccard_similarity = len(query_terms.intersection(content_terms)) / len(query_terms.union(content_terms))

        recommendations.append((index, jaccard_similarity))

    # Ordena por similitud descendente
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Retorna las mejores recomendaciones
    return recommendations[:num_recommendations]

# Ejemplo de uso
query_terms = {'vegan','burger'}
recommendations = get_recommendations_by_terms(dataset_stem, query_terms)

In [63]:
recommendations

[(1266345, 0.5),
 (516843, 0.3333333333333333),
 (516908, 0.3333333333333333),
 (518509, 0.3333333333333333),
 (519766, 0.3333333333333333),
 (529796, 0.3333333333333333),
 (531212, 0.3333333333333333),
 (604739, 0.3333333333333333),
 (932786, 0.3333333333333333),
 (517273, 0.25)]

In [64]:
element = dataset_stem.loc[1266345]
print(element)

text_stem         love thier vegan burger
ex_categories    Southern restaurant (US)
Name: 1266345, dtype: object


In [65]:
for index, similarity in recommendations:
    element = df.loc[index]
    print(f"Index: {index}, Similarity: {similarity}")
    # Muestra otros datos específicos del elemento si es necesario
    print(element)



Index: 1266345, Similarity: 0.5
business_id                                                   None
rating                                                         5.0
text                                    Loved thier Vegan Burgers!
name                               House of Blues Restaurant & Bar
address          House of Blues Restaurant & Bar, 1490 E Buena ...
latitude                                                 28.371029
longitude                                               -81.521959
postal_code                                                  32830
ex_categories                             Southern restaurant (US)
state_name                                                 Florida
city                                                          None
Name: 1266345, dtype: object
Index: 516843, Similarity: 0.3333333333333333
business_id                                                   None
rating                                                         5.0
text                  

In [71]:
def get_non_related_records(df, query_terms, threshold=0.2):
    non_related_records = []

    for index, row in df.iterrows():
        content_terms = set(row['text_stem'].split())  # Suponiendo que 'content' contiene las palabras clave del contenido

        # Calcula la similitud de Jaccard entre los términos de consulta y los términos del contenido
        jaccard_similarity = len(query_terms.intersection(content_terms)) / len(query_terms.union(content_terms))

        if jaccard_similarity < threshold:
            non_related_records.append((index, jaccard_similarity))

    # Ordena por similitud ascendente (menor similitud primero)
    non_related_records.sort(key=lambda x: x[1])

    return non_related_records

# Ejemplo de uso
query_terms_o = {'vegan','burger'}
recommendations_o = get_non_related_records(dataset_stem, query_terms)


In [73]:
len(recommendations_o)

12312

In [72]:
for index, similarity in recommendations_o:
    element = df.loc[index]
    print(f"Index: {index}, Similarity: {similarity}")
    # Muestra otros datos específicos del elemento si es necesario
    print(element)

Se han truncado las últimas 5000 líneas del flujo de salida.
address          House of Blues Restaurant & Bar, 1490 E Buena ...
latitude                                                 28.371029
longitude                                               -81.521959
postal_code                                                  32830
ex_categories                             Southern restaurant (US)
state_name                                                 Florida
city                                                          None
Name: 1264939, dtype: object
Index: 1264962, Similarity: 0.0
business_id                                                   None
rating                                                         5.0
text             Love the outdoor patio and nightly music. The ...
name                               House of Blues Restaurant & Bar
address          House of Blues Restaurant & Bar, 1490 E Buena ...
latitude                                                 28.371029
longitu